<a href="https://colab.research.google.com/github/Khushmeet-patil/Patient_report_summariser/blob/main/doctor_doc_summariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pymongo ngrok pyngrok langchain_groq streamlit PyPDF2 langchain_community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import os
import tempfile
import time
from datetime import datetime
import pkg_resources
import streamlit as st
import PyPDF2
from langchain_groq import ChatGroq

# Function to safely handle rerun based on Streamlit version
def safe_rerun():
    """Safely rerun the app based on Streamlit version"""
    try:
        st_version = pkg_resources.get_distribution('streamlit').version
        if pkg_resources.parse_version(st_version) >= pkg_resources.parse_version('1.27.0'):
            st.rerun()
        else:
            st.experimental_rerun()
    except Exception as e:
        try:
            st.rerun()
        except:
            try:
                st.experimental_rerun()
            except:
                st.error("Unable to refresh the page. Please refresh manually (F5 or Ctrl+R)")

# Set page configuration
st.set_page_config(
    page_title="Medical Document Analyzer",
    page_icon="🏥",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better UI and chat visibility
st.markdown("""
    <style>
    .main {
        padding: 2rem;
    }
    .stButton > button {
        width: 100%;
        border-radius: 5px;
        height: 3em;
    }
    .chat-message {
        padding: 1.5rem;
        border-radius: 0.5rem;
        margin-bottom: 1rem;
        display: flex;
        flex-direction: column;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    .chat-message.user {
        background-color: #2196F3;
        color: white;
    }
    .chat-message.user strong {
        color: #E3F2FD;
    }
    .chat-message.assistant {
        background-color: #F5F5F5;
        color: #333333;
        border-left: 4px solid #2196F3;
    }
    .chat-message.assistant strong {
        color: #1976D2;
    }
    .file-uploader {
        padding: 2rem;
        border: 2px dashed #ccc;
        border-radius: 10px;
        text-align: center;
    }
    .stTextInput>div>div>input {
        background-color: white;
        color: #333333;
    }
    </style>
    """, unsafe_allow_html=True)

# Initialize session state
if 'patient_records' not in st.session_state:
    st.session_state.patient_records = {}
if 'llm' not in st.session_state:
    # Initialize the LLM
    os.environ['GROQ_API_KEY'] = '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'
    st.session_state.llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0,
        max_tokens=8000,
        timeout=None,
        max_retries=2
    )

def extract_text_from_pdf(pdf_file):
    """Extract text from a PDF file."""
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        st.error(f"Error extracting text from PDF: {str(e)}")
        return None

def generate_summary(text):
    """Generate a summary using the LLM."""
    try:
        messages = [
            ("system", """You are an AI medical assistant specialized in analyzing medical documents.
            Please provide a concise summary focusing on key medical findings, diagnoses,
            treatments, and recommendations. Use medical terminology appropriately."""),
            ("human", text)
        ]
        response = st.session_state.llm.invoke(messages)
        return response.content
    except Exception as e:
        st.error(f"Error generating summary: {str(e)}")
        return None

def process_uploaded_file(uploaded_file):
    """Process a single uploaded PDF file."""
    if uploaded_file.type != "application/pdf":
        st.error(f"{uploaded_file.name} is not a PDF file.")
        return None

    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(uploaded_file.getvalue())
        text = extract_text_from_pdf(temp_file.name)
        os.unlink(temp_file.name)

        if text:
            summary = generate_summary(text)
            if summary:
                return {
                    'filename': uploaded_file.name,
                    'text': text,
                    'summary': summary,
                    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
    return None

def chat_with_docs(question, context):
    """Chat with the documents using the LLM."""
    try:
        messages = [
            ("system", """You are an AI medical assistant. Use the provided medical document
            context to answer questions. If the answer cannot be found in the context,
            clearly state that. Always maintain medical accuracy and professionalism."""),
            ("human", f"Context: {context}\n\nQuestion: {question}")
        ]
        response = st.session_state.llm.invoke(messages)
        return response.content
    except Exception as e:
        st.error(f"Error processing question: {str(e)}")
        return None

# Sidebar
with st.sidebar:
    st.image("https://via.placeholder.com/150?text=Medical+Logo", width=150)
    st.title("Patient Records")

    # File uploader
    uploaded_file = st.file_uploader(
        "Upload Patient Record (PDF)",
        type=["pdf"],
        accept_multiple_files=False
    )

    if uploaded_file:
        if st.button("Process Record"):
            with st.spinner("Processing record..."):
                patient_record = process_uploaded_file(uploaded_file)
                if patient_record:
                    st.session_state.patient_records[uploaded_file.name] = patient_record
                    st.success(f"Processed: {uploaded_file.name}")

    # Show processed files
    if st.session_state.patient_records:
        st.subheader("Patient Records")
        for filename in st.session_state.patient_records:
            st.text(f"📄 {filename}")

# Main content area
st.title("Medical Document Analyzer")

# Tabs for different views
tab1, tab2 = st.tabs(["Patient Summaries", "Chat Interface"])

with tab1:
    if st.session_state.patient_records:
        for filename, data in st.session_state.patient_records.items():
            with st.expander(f"Summary: {filename}"):
                st.write(f"**Processed on:** {data['timestamp']}")
                st.write("**Summary:**")
                st.write(data['summary'])

                if st.button(f"View Full Text ({filename})", key=f"full_text_{filename}"):
                    st.write("**Full Text:**")
                    st.text_area("", data['text'], height=300, key=f"text_{filename}")
    else:
        st.info("Upload and process patient records to see summaries here.")

with tab2:
    st.subheader("Chat with Patient Records")

    if st.session_state.patient_records:
        # Display chat history
        if 'chat_history' not in st.session_state:
            st.session_state.chat_history = []

        for message in st.session_state.chat_history:
            with st.container():
                st.markdown(f"""
                    <div class="chat-message {message['role']}">
                        <div><strong>{'You' if message['role'] == 'user' else 'Assistant'}:</strong></div>
                        <div style="margin-top: 0.5rem;">{message['content']}</div>
                    </div>
                    """, unsafe_allow_html=True)

        # Chat input with error handling
        question = st.text_input("Ask a question about your patient records:", key="chat_input")
        if question:
            if st.button("Send", key="send_button"):
                # Combine all patient record contexts
                context = "\n\n".join([
                    f"Patient Record: {filename}\n{data['text']}"
                    for filename, data in st.session_state.patient_records.items()
                ])

                with st.spinner("Thinking..."):
                    response = chat_with_docs(question, context)

                if response:
                    # Add to chat history
                    st.session_state.chat_history.append({
                        'role': 'user',
                        'content': question
                    })
                    st.session_state.chat_history.append({
                        'role': 'assistant',
                        'content': response
                    })
                    # Use the safe rerun function
                    safe_rerun()
    else:
        st.info("Upload and process patient records to start chatting.")

# Footer
st.markdown("---")
st.markdown(
    "Made with ❤️ for healthcare professionals. "
    "Upload medical documents securely and get AI-powered insights."
)

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501